In [75]:
import torch
import torchmetrics
import torch.nn as nn

import numpy as np
import pandas as pd

df_train = pd.read_csv("train.csv")

print(df_train.iloc[:,1])

df_test = pd.read_csv("test.csv")

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [135]:
import torch
X_train = df_train.iloc[:,df_train.columns != "Survived"]
y_train = df_train.iloc[:,1]

X_test = df_test.iloc[:]

X_train, y_train


(     PassengerId  Pclass                                               Name  \
 0              1       3                            Braund, Mr. Owen Harris   
 1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
 2              3       3                             Heikkinen, Miss. Laina   
 3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
 4              5       3                           Allen, Mr. William Henry   
 ..           ...     ...                                                ...   
 886          887       2                              Montvila, Rev. Juozas   
 887          888       1                       Graham, Miss. Margaret Edith   
 888          889       3           Johnston, Miss. Catherine Helen "Carrie"   
 889          890       1                              Behr, Mr. Karl Howell   
 890          891       3                                Dooley, Mr. Patrick   
 
         Sex   Age  SibSp  Parch      

In [77]:
class ModelV1 (nn.Module):
    def __init__(self, 
                 in_features: int,
                 out_features: int,
                 hidden_units: int
                 ):
        super().__init__()

        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=in_features,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,
                      out_features=out_features)
        )

    def forward(self, x):
        x = self.layer_stack(x)
        return x
    
model_0 = ModelV1(in_features=11, hidden_units=32, out_features=1)

In [103]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from timeit import default_timer as timer

train_dataloader = DataLoader(dataset=X_train[0:497],
                              batch_size=16,
                              shuffle=True)

In [104]:
# Setup loss and optimizer fns

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(lr=0.01,
                             params=model_0.parameters())
acc_metric = torchmetrics.Accuracy(task="binary", num_classes=2)

In [134]:
train_loss, train_acc = 0, 0
    
epochs = 100

model_0.train()

for epoch in tqdm(range(epochs)):
    y_pred = model_0(X_train)

    loss = loss_fn(y_pred, y_train)
    train_loss += loss
    train_acc += acc_metric(y_pred, y_train) * 100

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train acc: {train_acc} | Train loss: {train_loss}")


  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray